In [1]:
import torch
import arviz as az
from scipy.optimize import curve_fit
from bokeh.plotting import figure, show,output_file, save
from bokeh.transform import factor_cmap, factor_mark
from bokeh.palettes import Spectral
from bokeh.models import Slope, Div
from bokeh.io import curdoc,output_notebook,export_png
from bokeh.layouts import column,gridplot
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from bokeh.models import Band, ColumnDataSource
import pandas as pd
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from Plotting import *

In [2]:
train=pd.read_excel(r"../data/chem_train.xlsx")
test=pd.read_excel(r"../data/chem_test.xlsx")

In [3]:
gpr_no2=torch.load("../models/no2_seasonality",weights_only=False,map_location='cuda')

In [4]:
gpr_nox=torch.load("../models/nox_seasonality",weights_only=False,map_location='cuda')

In [5]:
train_predict=train.copy()
train=train.loc[(train.corrected_week>0)&(train.corrected_week<53),:]
test_predict=test.copy()
test=test.loc[(test.corrected_week>0)&(test.corrected_week<53),:]

In [6]:
train

,Unnamed: 0.1,cpnr,Unnamed: 0,maalnr,constrution_year,Area,floor_level,powerplant,road,motorway,...,gastove,fireplace,visit,no2,nox,formaldehyd,acetald,acetone,sample_time,corrected_week
0,0,104,246.0,1.0,1970.0,156.0,0.0,0.0,17893.5,10.0,...,2.0,150.0,1,7.8,21.3,25.900000,23.300000,54.100000,76,46.000000
1,1,104,248.0,3.0,1970.0,156.0,0.0,0.0,17893.5,10.0,...,0.0,220.0,3,6.5,14.6,26.498496,2.625317,6.299715,72,43.857143
2,2,104,248.0,3.0,1970.0,156.0,0.0,0.0,17893.5,10.0,...,0.0,220.0,4,4.6,4.5,31.393703,2.727302,7.705821,71,16.714286
3,3,257,621.0,1.0,1996.0,110.0,0.0,0.0,4280.3,10.8,...,5.0,90.0,1,5.5,5.8,26.783053,12.992725,16.256963,70,31.142857
4,4,257,623.0,3.0,1996.0,110.0,0.0,0.0,4280.3,10.8,...,18.0,190.0,3,5.8,NaN,15.803975,1.531092,1.307573,71,27.428571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616,0,346,837.0,1.0,1991.0,91.0,1.0,947.9,22543.7,2582.3,...,0.0,10.0,1,12.8,30.2,17.057017,10.478731,18.147374,70,52.442857
642,0,243,592.0,2.0,1922.0,122.0,0.0,1611.6,24983.2,1644.2,...,180.0,90.0,2,9.3,31.2,26.020930,17.553766,9.855742,65,52.871429
665,0,24,43.0,2.0,1905.0,194.0,0.0,4102.2,8142.9,4133.9,...,3.0,0.0,2,5.3,21.2,12.000000,17.300000,15.200000,71,52.728571
697,0,357,867.0,1.0,1983.0,95.0,1.0,4198.8,25676.3,369.3,...,50.0,0.0,1,9.2,23.7,15.517275,2.543006,1.312857,117,52.585714


In [7]:
mean_no2_train,data_no2_train=roll_week(train,1000,2,particle=["no2"])
mean_nox_train,data_nox_train=roll_week(train,1000,2,particle=["nox"])
mean_no2_test,data_no2_test=roll_week(test,1000,2,particle=["no2"])
mean_no2_test,data_nox_test=roll_week(test,1000,2,particle=["nox"])

In [8]:
plot_no2,model_no2=gaussian_plot(np.linspace(0,52.5,1000),gpr_no2) 
plot_nox,model_nox=gaussian_plot(np.linspace(0,52.5,1000),gpr_nox) 

In [9]:
r2_no2,r2_nox=r2_score(data_no2_train.y,model_no2.y),r2_score(data_nox_train.y,model_nox.y)
r2_no2_test,r2_nox_test=r2_score(data_no2_test.y,model_no2.y),r2_score(data_nox_test.y,model_nox.y)

In [10]:
output_notebook()
TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,examine,help"

p = figure(x_range=(0, 53),y_range=(0,15));
p.title.text = r"$$NO_2$$ seasonality";
p.xgrid.grid_line_color=None;
p.ygrid.grid_line_alpha=0.5;
p.line(data_no2_train.index, data_no2_train.y, line_width=3,color="green",legend_label="Train roll mean R2 "+str(round(r2_no2,2)));
p.line(data_no2_test.index, data_no2_test.y, line_width=3,color="orange",legend_label="Test roll mean R2 "+str(round(r2_no2_test,2)));
p.line(model_no2.index, model_no2.y, line_width=3,color="red",legend_label="Gaussion model ");
p.scatter(train.corrected_week, y=train.no2, color="blue", marker="dot", size=20, alpha=0.4,legend_label="raw points");
band = Band(base="index", lower="lower", upper="upper",source=mean_no2_train, fill_color="red", line_color="black",fill_alpha=0.2);
band1 = Band(base="index", lower="lower", upper="upper",source=plot_no2,fill_alpha=0.5, fill_color="blue", line_color="black");
p.yaxis.axis_label_orientation  = 0
p.add_layout(band);
p.add_layout(band1);
p.xaxis.axis_label = r'$$Week \ of \ the \ year$$';
p.yaxis.axis_label = r'$$\frac{\mu g}{m^3} $$';


p1 = figure(x_range=(0, 53),y_range=(0,60));
p1.title.text = r"NO_x seasonality";
p1.xgrid.grid_line_color=None;
p1.ygrid.grid_line_alpha=0.5;
p1.line(data_nox_train.index, data_nox_train.y, line_width=3,color="green",legend_label="Train roll mean R2 "+str(round(r2_nox,2)));
p1.line(data_nox_test.index, data_nox_test.y, line_width=3,color="orange",legend_label="Test roll mean R2 "+str(round(r2_nox_test,2)));
p1.line(model_nox.index, model_nox.y, line_width=3,color="red",legend_label="Gaussion model ");
p1.scatter(train.corrected_week, y=train.nox, color="blue", marker="dot", size=20, alpha=0.4,legend_label="raw points");
band2 = Band(base="index", lower="lower", upper="upper",source=mean_nox_train, fill_color="red", line_color="black",fill_alpha=0.2);
band3 = Band(base="index", lower="lower", upper="upper",source=plot_nox,fill_alpha=0.5, fill_color="blue", line_color="black");
p1.yaxis.axis_label_orientation  = 0
p1.add_layout(band2);
p1.add_layout(band3);
p1.xaxis.axis_label = r'$$Week \ of \ the \ year$$';
p1.yaxis.axis_label = r'$$\frac{\mu g}{m^3} $$';
grid =  gridplot([[p, p1]], width=500, height=500);

show(column(grid));

Loading BokehJS ...